In [ ]:
import ScraperFC as sfc  # LEAVE THIS!
import pandas as pd

In [ ]:

ss = sfc.Sofascore()

# 1. League-level stats (NO player IDs by design)
df = ss.scrape_player_league_stats(
    year="25/26",
    league="Turkish Super Lig",
    accumulation="total"
)

InvalidLeagueException: Trendyol Super Lig is not a valid league for Sofascore. Valid leagues are ['Champions League', 'Europa League', 'Europa Conference League', 'EPL', 'La Liga', 'Bundesliga', 'Serie A', 'Ligue 1', 'Turkish Super Lig', 'Argentina Liga Profesional', 'Argentina Copa de la Liga Profesional', 'Liga 1 Peru', 'Copa Libertadores', 'MLS', 'USL Championship', 'USL1', 'USL2', 'Saudi Pro League', 'World Cup', 'Euros', 'Gold Cup', "Women's World Cup"]

In [26]:
df.to_csv("tackles.csv", index=False)


In [27]:
import requests
from datetime import datetime, timezone

In [28]:
import requests
from datetime import datetime, timezone


def get_player_age_and_country(player_id: int) -> tuple[int, str | None]:
    url = f"https://api.sofascore.com/api/v1/player/{player_id}"
    headers = {"User-Agent": "Mozilla/5.0"}

    r = requests.get(url, headers=headers)
    r.raise_for_status()

    data = r.json()
    player_data = data["player"]

    dob_ts = player_data.get("dateOfBirthTimestamp")
    if dob_ts is None:
        raise ValueError("dateOfBirthTimestamp not found")

    dob = datetime.fromtimestamp(dob_ts, tz=timezone.utc)
    today = datetime.now(tz=timezone.utc)

    age = today.year - dob.year
    if (today.month, today.day) < (dob.month, dob.day):
        age -= 1

    country = None
    country_obj = player_data.get("country")
    if isinstance(country_obj, dict):
        country = (
            country_obj.get("name")
            or country_obj.get("alpha2")
            or country_obj.get("code")
        )
    if not country and "nationality" in player_data:
        country = player_data["nationality"]

    return age, country


Could not get age for player_id 1121174: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/1121174
Could not get age for player_id 171625: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/171625
Could not get age for player_id 927649: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/927649
Could not get age for player_id 923888: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/923888
Could not get age for player_id 855672: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/855672
Could not get age for player_id 1418644: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/1418644
Could not get age for player_id 859783: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/859783
Could not get age for player_id 310068: 403 Client Error: challenge for url: https://api.sofascore.com/api/v1/player/310068
Coul

In [ ]:
print(df['country'][df['player']=='Marco Asensio'])


KeyError: 'country'

In [ ]:
# Get all player ages and countries and left join to df
player_ids = df['player id'].unique()

player_info = []
for player_id in player_ids:
    try:
        # Convert numpy int64 to native Python int
        player_id_int = int(player_id)
        age, country = get_player_age_and_country(player_id_int)
        player_info.append({
            'player id': player_id_int,
            'age': age,
            'country': country
        })
    except Exception as e:
        # Skip players where age cannot be retrieved
        print(f"Could not get info for player_id {player_id}: {e}")
        continue

info_df = pd.DataFrame(player_info)

# Left join ages and countries to df
df = df.merge(info_df, on='player id', how='left')

print(f"Added ages and countries for {len(info_df)} players")
print(f"Total rows in df: {len(df)}")
print(f"Rows with age: {df['age'].notna().sum()}")
print(f"Rows with country: {df['country'].notna().sum()}")

Could not get info for player_id 2266453: cannot unpack non-iterable int object
Could not get info for player_id 913985: cannot unpack non-iterable int object
Could not get info for player_id 1065508: cannot unpack non-iterable int object
Could not get info for player_id 361004: cannot unpack non-iterable int object
Could not get info for player_id 857738: cannot unpack non-iterable int object
Could not get info for player_id 1063942: cannot unpack non-iterable int object
Could not get info for player_id 187205: cannot unpack non-iterable int object
Could not get info for player_id 293519: cannot unpack non-iterable int object
Could not get info for player_id 827418: cannot unpack non-iterable int object
Could not get info for player_id 1128844: cannot unpack non-iterable int object
Could not get info for player_id 359332: cannot unpack non-iterable int object
Could not get info for player_id 845792: cannot unpack non-iterable int object
Could not get info for player_id 758608: cannot 

KeyError: 'player id'

In [ ]:
df.to_csv("tackles_joined.csv", index=False)
